In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
cmap = plt.cm.tab20
from sklearn.model_selection import train_test_split
import tensorflow as tf
layers = tf.keras.layers

In [2]:
data = pd.read_csv("dataset/german_credit_data_withrisk.csv", index_col=0)

In [3]:
# data = data[["Credit amount", "Age", "Duration", "Purpose", "Risk"]]

In [4]:
data.head()

,Age,Sex,Job,Housing,Saving accounts,Checking account,Credit amount,Duration,Purpose,Risk
0,67,male,2,own,NaN,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,NaN,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad


In [5]:
num_impute = "mean" # One of ["mean", "zero", "infinity"]
cat_impute = "none" # One of ["mode", "none"]

In [6]:
job_index2word = {
    0: "unskilled and non-resident", 
    1: "unskilled and resident", 
    2: "skilled", 
    3: "highly skilled"
}
def assign_job_type(col):
    return job_index2word[col]

In [7]:
data.Job = data.Job.apply(assign_job_type)

In [8]:
cat_cols = ['Sex', 'Job', 'Housing', 'Saving accounts', 'Checking account', 'Purpose']
num_cols = ['Age', 'Credit amount', 'Duration']

In [9]:
def impute_with_mean(df):
    out = pd.DataFrame(df)
    for col in df.columns:
        if col in num_cols:
            out.loc[out[col].isna(), col] = df[col].mean()
    return out
def impute_with_zero(df):
    out = pd.DataFrame(df)
    for col in df.columns:
        if col in num_cols:
            out.loc[out[col].isna(), col] = 0.0
    return out


def impute_with_infinity(df):
    out = pd.DataFrame(df)
    for col in df.columns:
        if col in num_cols:
            out.loc[out[col].isna(), col] = float("inf")
    return out
def impute_with_mode(df):
    out = pd.DataFrame(df)
    for col in df.columns:
        if col in cat_cols:
            out.loc[out[col].isna(), col] = df[col].mode().iat[0]
    return out
def impute_with_none(df):
    out = pd.DataFrame(df)
    for col in df.columns:
        if col in cat_cols:
            out.loc[out[col].isna(), col] = "None"
    return out

In [10]:
def get_impute_function(name):
    assert name in ["mean", "zero", "infinity", "mode", "none"]
    if name == "mean":
        return impute_with_mean
    elif name == "zero":
        return impute_with_zero
    elif name == "infinity":
        return impute_with_infinity
    elif name == "mode":
        return impute_with_mode
    else:
        return impute_with_none

In [11]:
def impute_missing_values(df, num_impute, cat_impute):
    num_impute_function = get_impute_function(num_impute)
    cat_impute_function = get_impute_function(cat_impute)
    new_df = num_impute_function(df)
    new_df = cat_impute_function(df)
    return new_df

In [12]:
data.isna().sum()

Age                   0
Sex                   0
Job                   0
Housing               0
Saving accounts     183
Checking account    394
Credit amount         0
Duration              0
Purpose               0
Risk                  0
dtype: int64

In [13]:
data = impute_missing_values(data, num_impute, cat_impute)

In [14]:
data.isna().sum()

Age                 0
Sex                 0
Job                 0
Housing             0
Saving accounts     0
Checking account    0
Credit amount       0
Duration            0
Purpose             0
Risk                0
dtype: int64

In [15]:
(data.Risk == "good").sum()

700

In [16]:
(data.Risk == "bad").sum()

300

In [17]:
def cat2onehot(df):
    output = df.copy()
    for col in cat_cols:
        if col in df.columns:
            dummies = pd.get_dummies(df[col], prefix="", prefix_sep = col+"_")
            output = pd.concat([dummies, output], axis=1)
  
            output.drop(col, axis=1, inplace=True)
      
    return output

In [18]:
data = cat2onehot(data)

In [19]:
data.columns

Index(['Purpose_business', 'Purpose_car', 'Purpose_domestic appliances',
       'Purpose_education', 'Purpose_furniture/equipment', 'Purpose_radio/TV',
       'Purpose_repairs', 'Purpose_vacation/others', 'Checking account_None',
       'Checking account_little', 'Checking account_moderate',
       'Checking account_rich', 'Saving accounts_None',
       'Saving accounts_little', 'Saving accounts_moderate',
       'Saving accounts_quite rich', 'Saving accounts_rich', 'Housing_free',
       'Housing_own', 'Housing_rent', 'Job_highly skilled', 'Job_skilled',
       'Job_unskilled and non-resident', 'Job_unskilled and resident',
       'Sex_female', 'Sex_male', 'Age', 'Credit amount', 'Duration', 'Risk'],
      dtype='object')

In [20]:
def normalize_numerical(df):
    output = df.copy()
    norm_params = {}
    for col in num_cols:
        mean = df[col].mean()
        std = df[col].std()
        
        output[col] = (df[col] - mean)/std
        norm_params[col] = {"mean":mean, "std":std}
    return output, norm_params

In [21]:
data.head()

,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Checking account_None,Checking account_little,...,Job_highly skilled,Job_skilled,Job_unskilled and non-resident,Job_unskilled and resident,Sex_female,Sex_male,Age,Credit amount,Duration,Risk
0,0,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,1,67.0,1169.0,6.0,good
1,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,22.0,5951.0,48.0,bad
2,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,1,49.0,2096.0,12.0,good
3,0,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,1,45.0,7882.0,42.0,good
4,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,53.0,4870.0,24.0,bad


In [22]:
data, norm_params = normalize_numerical(data)

In [23]:
data.head()

,Purpose_business,Purpose_car,Purpose_domestic appliances,Purpose_education,Purpose_furniture/equipment,Purpose_radio/TV,Purpose_repairs,Purpose_vacation/others,Checking account_None,Checking account_little,...,Job_highly skilled,Job_skilled,Job_unskilled and non-resident,Job_unskilled and resident,Sex_female,Sex_male,Age,Credit amount,Duration,Risk
0,0,0,0,0,0,1,0,0,0,1,...,0,1,0,0,0,1,2.765073,-0.744759,-1.235859,good
1,0,0,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,-1.190808,0.949342,2.247070,bad
2,0,0,0,1,0,0,0,0,1,0,...,0,0,0,1,0,1,1.182721,-0.416354,-0.738298,good
3,0,0,0,0,1,0,0,0,0,1,...,0,1,0,0,0,1,0.831087,1.633430,1.749509,good
4,0,1,0,0,0,0,0,0,0,1,...,0,1,0,0,0,1,1.534354,0.566380,0.256825,bad


In [24]:
X = data[data.columns[:-1]].values
y = (data.Risk == "good").astype(float)

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 123)

In [107]:
model = tf.keras.Sequential([
    layers.Dense(32, activation="relu", kernel_regularizer="l2"),
    layers.Dropout(0.25),
    layers.Dense(64, activation="relu", kernel_regularizer="l2"),
    layers.Dense(128, activation="relu", kernel_regularizer="l2"),
    layers.Dense(256, activation="relu", kernel_regularizer="l2"),
    layers.Dense(1, activation="sigmoid")
])

In [108]:
model.compile(loss="binary_crossentropy", optimizer=tf.keras.optimizers.Adam(1e-3), metrics=["accuracy"])

In [109]:
model.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=100)

Epoch 1/100
25/25 [==============================] - 0s 19ms/step - loss: 3.3128 - accuracy: 0.6950 - val_loss: 2.7247 - val_accuracy: 0.7000
Epoch 2/100
25/25 [==============================] - 0s 7ms/step - loss: 2.2964 - accuracy: 0.7000 - val_loss: 1.9358 - val_accuracy: 0.7000
Epoch 3/100
25/25 [==============================] - 0s 12ms/step - loss: 1.6635 - accuracy: 0.7025 - val_loss: 1.4528 - val_accuracy: 0.7100
Epoch 4/100
25/25 [==============================] - 0s 13ms/step - loss: 1.2704 - accuracy: 0.7250 - val_loss: 1.1527 - val_accuracy: 0.7350
Epoch 5/100
25/25 [==============================] - 0s 14ms/step - loss: 1.0259 - accuracy: 0.7337 - val_loss: 0.9751 - val_accuracy: 0.7300
Epoch 6/100
25/25 [==============================] - 0s 17ms/step - loss: 0.8830 - accuracy: 0.7400 - val_loss: 0.8635 - val_accuracy: 0.7250
Epoch 7/100
25/25 [==============================] - 0s 10ms/step - loss: 0.7919 - accuracy: 0.7312 - val_loss: 0.7930 - val_accuracy: 0.7200
Epoch 8

25/25 [==============================] - 0s 13ms/step - loss: 0.5017 - accuracy: 0.8112 - val_loss: 0.6362 - val_accuracy: 0.7100
Epoch 59/100
25/25 [==============================] - 0s 15ms/step - loss: 0.5096 - accuracy: 0.7788 - val_loss: 0.6300 - val_accuracy: 0.7150
Epoch 60/100
25/25 [==============================] - 0s 15ms/step - loss: 0.5070 - accuracy: 0.7987 - val_loss: 0.6593 - val_accuracy: 0.7350
Epoch 61/100
25/25 [==============================] - 0s 14ms/step - loss: 0.5044 - accuracy: 0.8037 - val_loss: 0.6428 - val_accuracy: 0.7250
Epoch 62/100
25/25 [==============================] - 0s 15ms/step - loss: 0.4999 - accuracy: 0.8050 - val_loss: 0.6361 - val_accuracy: 0.7150
Epoch 63/100
25/25 [==============================] - 0s 14ms/step - loss: 0.5071 - accuracy: 0.7925 - val_loss: 0.6490 - val_accuracy: 0.7250
Epoch 64/100
25/25 [==============================] - ETA: 0s - loss: 0.5101 - accuracy: 0.79 - 0s 12ms/step - loss: 0.5037 - accuracy: 0.7950 - val_loss: 